# Tutorial 4: How to get all mitotic cells in one cell line

The quilt data package (hipsc_single_cell_image_dataset) contains 216062 single cells segmented from 18186 field-of-view (FOV) with selected features calculated for each cell. In this tutorial, we will show **how to get all mitotic cells in one cell line**, which can be used for observing or analyzing the shape of a structure during mitosis.

In [ ]:
######### FOR google COLAB user only #########
### install necessary packages if in colab ###
##############################################

############################################################
### make sure to restart runtime after running this step ###
############################################################
def run_subprocess_command(cmd):
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    for line in process.stdout:
        print(line.decode().strip())

import sys, subprocess

IN_COLAB = "google.colab" in sys.modules
colab_requirements = [
    "pip install urllib3==1.25.4",
    "pip install PyYAML==5.1",
    "pip install quilt3",
    "pip install aicsimageio",
]
if IN_COLAB:
    for i in colab_requirements:
        run_subprocess_command(i)

In [ ]:
import pandas as pd
import numpy as np
import quilt3
from pathlib import Path
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

## Step 1: connect to the data storage

In [ ]:
# connect to quilt
pkg = quilt3.Package.browse("aics/hipsc_single_cell_image_dataset", registry="s3://allencell")
meta_df = pkg["metadata.csv"]()

In [ ]:
# a quick look at what are the columns 
print(meta_df.columns)

## step 2: select one cell line

In [ ]:
target_cellline = "FBL"
data = meta_df.query("structure_name==@target_cellline")

In [ ]:
data["cell_stage"].unique()

In [ ]:
data_mitotic = data.query("cell_stage!='M0'")

In [ ]:
data_mitotic.shape

## step 3: download data

In [ ]:
# prepare file paths
save_path = Path("C:/projects/allen_cell_data/")
save_path.mkdir(exist_ok=True)

In [ ]:
# identify all unique FOVs
fov_list = data_mitotic["FOVId"].unique()

for fov_id in fov_list:
    # prepare the path to save this FOV
    raw_path = save_path / Path(str(fov_id))
    raw_path.mkdir(exist_ok=True)
    
    # get all mitotic cells in this FOV
    fov_mitotic = data_mitotic.query("FOVId==@fov_id")
    
    if len(fov_mitotic) < 2:
        continue
    
    # fetch raw image:
    fov_path = fov_mitotic["fov_path"].iloc[0]
    ch = fov_mitotic["ChannelNumberStruct"].iloc[0]
    subdir_name = fov_path.split("/")[0]
    file_name = fov_path.split("/")[1]
    local_fn = raw_path / f"original_{fov_id}_ch_{ch}.tiff"
    pkg[subdir_name][file_name].fetch(local_fn)
    
    # fetch fov segmentation:
    fov_seg_path = fov_mitotic["fov_seg_path"].iloc[0]
    subdir_name = fov_seg_path.split("/")[0]
    file_name = fov_seg_path.split("/")[1]
    seg_fn = raw_path / f"fov_segmentation_{fov_id}.tiff"
    pkg[subdir_name][file_name].fetch(seg_fn)
    
    # fetch structure segmentation
    struct_seg_path = fov_mitotic["struct_seg_path"].iloc[0]
    subdir_name = struct_seg_path.split("/")[0]
    file_name = struct_seg_path.split("/")[1]
    struct_seg_fn = raw_path / f"structure_segmentation_{fov_id}.tiff"
    pkg[subdir_name][file_name].fetch(struct_seg_fn)
    
    # read the cell segmentation
    reader = AICSImage(seg_fn)
    cell_seg = reader.get_image_data("ZYX", C=1, S=0, T=0)
    
    # get mask of each mitotic cell
    for row in fov_mitotic.itertuples():
        # get cell index
        c_id = row.this_cell_index
        
        # mitotic label
        m_lab = row.cell_stage
        
        # mask for this cell
        cell_seg_this_cell = cell_seg==c_id
        cell_seg_this_cell = cell_seg_this_cell.astype(np.uint8)

        # save the mask
        out_fn = raw_path / f"cell-{c_id}-{m_lab}_mask.tiff"
        with OmeTiffWriter(out_fn) as writer:
            writer.save(
                cell_seg_this_cell,
                dimension_order='ZYX'
            )